In [1]:
import requests
import json
import csv
import io
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth

In [2]:
def SubmitPMIDList(Pmid,Format,Bioconcept):

    json = {"pmids": Pmid}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
        
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
    res = r.json()
    
    pmid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['pmid'] = pmid
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [ ]:
def SubmitPMIDList(Pmcid,Format,Bioconcept):

    json = {"Pmcids": Pmcid}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
        
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
    res = r.json()
    
    pmcid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['Pmcid'] = Pmcid
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [3]:
Inputfile = "./pmcid"
Format = "biocjson"
Bioconcept = ""
res_json=[]

In [4]:
with io.open(Inputfile,'r',encoding="utf-8") as file_input:
    pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}

for i in pmidlist['pmids']:
    res_json.extend(SubmitPMIDList([i], Format, Bioconcept))    

In [5]:
class my_dictionary(dict): 
    
    def __init__(self): 
        self = dict() 
          
    def add(self, key, value): 
        self[key] = value 

dict_list = []

for data in res_json:
    dict_obj = my_dictionary() 
    
    for key, val in data.items():
        if type(val) == dict:    
            for k, v in val.items():
                dict_obj.add(k, v)
        elif type(val) == list:  
            for k, v in val[0].items():
                dict_obj.add(k, v)
        else:
            dict_obj.add(key, val)

            
    dict_list.append(dict_obj)

result = []
    
for item in dict_list:
    item = {key: item[key] for key in item.keys() & {'pmid', 'type', 'text'}}
    result.append(item)
    

In [6]:
result = [dict(t) for t in {tuple(d.items()) for d in result}]

In [7]:
data_file = open('./response.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

count = 0
 
for i in result:
    if count == 0:
        header = i.keys()
        csv_writer.writerow(header)
        count += 1
 
    csv_writer.writerow(i.values())
 
data_file.close()